In [1]:
import pandas as pd
import json
from collections import defaultdict
import requests
import os
import firebase_admin
from firebase_admin import credentials, auth
from firebase_admin import db, firestore
from tkinter import Tk,  filedialog
import datetime
import openpyxl

In [ ]:
cred = credentials.Certificate(r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\json_secret\logistec-firebase-adminsdk-mz58b-63b513fce5.json")

firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://logistec-default-rtdb.asia-southeast1.firebasedatabase.app/',\
    
})
firestore_db = firestore.client()
ref = db.reference('DHL_Shipment_Database')
id33 = ref.child('Stockout_33')
id49 = ref.child('Stockout_49')
id49_insure = ref.child('Stockout49_Insure')
id1607 = ref.child('Stockout_1607')
id32 = ref.child('Stockout_32')

In [14]:
data = pd.read_clipboard(sep='\t')

stockoutid = data['Stock Out (ID)'].astype(str)
branchid = data['Branch (ID)'].astype(str)
doc_date =  pd.to_datetime(data['Doc Date'], dayfirst=True).dt.strftime("%d/%m/%y")
branchto = data['Branch To (Name)']
clean_booking_id = data['Booking ID'].str.rstrip().str.replace("Booking-DHL ID : ", "")
id_concat = pd.DataFrame(stockoutid + "-" + branchid).rename(columns={0: "PHYID",}, inplace=False).PHYID.str.replace(",","")

frames = [id_concat, clean_booking_id, branchto, doc_date]

new_data = pd.concat(frames, axis=1).rename(columns={0: "PHYID", 1: "Booking ID", 2: "Branch To", 3: "Doc Date"})
new_new_data = new_data.set_index(['PHYID', 'Branch To (Name)', 'Doc Date'])
new_new_data = new_new_data['Booking ID'].str.split(" , " , expand=True).stack().reset_index(level=3, drop=True).reset_index(name='Tracking_index')
new_df = pd.DataFrame(new_new_data)
new_df['Tracking'] = new_df.loc[:, 'Tracking_index']
new_df = pd.DataFrame(new_df).set_index('Tracking')
#new_new_data = pd.concat(phyid_col, booking_id_col, branch_to_col).set_index('Booking ID')
#new_new_data = new_data.set_index(['Stock Out (ID)', 'Branch (ID)'])
#new_new_data = new_data['Booking ID'].str.split(' , ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='Booking-ID')
#new_new_data = pd.DataFrame(new_new_data,)
nested_dict = lambda: defaultdict(dict)
nested = nested_dict()

for row in new_df.itertuples():
    nested[row.Index] = {'PHYID' : row.PHYID, 'Branch To' : row[2], 'Tracking' : row[4],'Doc Date' : row[3]}

In [ ]:
for data in nested.values():
    try:
        print(data["Branch To"])
        break
    except KeyError:
        pass

for data2 in nested.values():
    print(data2) 


<H1>ID33 Head Office - Insure</H1>

In [6]:
#firestore
today = datetime.datetime.today().strftime("%d-%m-%Y")
id33 = firestore_db.collection('DHL_shipments_data').document('ID33: คลัง Service Headoffice - Insure')
for data in nested.values():
    branch = data["Branch To"]
if branch == 'ID33: คลัง Service Headoffice' or 'ID33: คลังสินค้า Service ฝั่ง Insure':
    try:
        id33.set(nested,merge=True)
    except KeyError:
        pass
    print("Data Sent")


Data Sent


In [ ]:
#RTDB
id33.update(nested)

<H1>ID49</h1>

In [8]:
today = datetime.datetime.today().strftime("%d-%m-%Y")
id49 = firestore_db.collection('DHL_shipments_data').document('ID49: สินค้าโยกเข้าคลัง')
for data in nested.values():
    branch = data["Branch To"]
if branch == "ID49 : สินค้าโยกเข้าคลัง (01/10/2008)":
    try:
        id49.set(nested,merge=True)
    except KeyError:
        pass
    print("Data Sent")


Data Sent


In [ ]:
id49.update(nested)

<H1>ID49 - Insure</h1>

In [10]:
today = datetime.datetime.today().strftime("%d-%m-%Y")
id49_insure = firestore_db.collection('DHL_shipments_data').document('ID49: สินค้า Trade-in')
for data in nested.values():
    branch = data["Branch To"]
if branch == 'ID49 : สินค้า Trade in':
    try:
        id49_insure.set(nested,merge=True)
    except KeyError:
        pass
    print("Data Sent")


Data Sent


In [ ]:
id49_insure.update(nested)

<H1>ID1607</H1>

In [12]:
today = datetime.datetime.today().strftime("%d-%m-%Y")
id1607 = firestore_db.collection('DHL_shipments_data').document('ID1607: สินค้า Spare ทำคืนลูกค้า')
for data in nested.values():
    branch = data["Branch To"]
if branch == "ID1607 : คลังสินค้า Service Headoffice (Spare)":
    try:
        id1607.set(nested,merge=True)
    except KeyError:
        pass
    print("Data Sent")


Data Sent


In [ ]:
id1607.update(nested)

<H1>ID32</>

In [15]:
today = datetime.datetime.today().strftime("%d-%m-%Y")
id32 = firestore_db.collection('DHL_shipments_data').document('ID32 : คลัง Service เครื่องใช้ไฟฟ้า Index')
for data in nested.values():
    branch = data["Branch To"]
if branch == 'ID32 : คลัง Service เครื่องใช้ไฟฟ้า Index':
    try:
        id32.set(nested,merge=True)
        #id32.set({})
    except KeyError:
        pass
    print("Data Sent")

Data Sent


<H1>unused (for now)</h1>

In [ ]:


with open("shipment_data_1.json", "w+",) as jsonfile:
    json.dump(nested, jsonfile, indent=3)
    #write to my firebase



<H1>Send data through email</H1>

In [39]:
today = datetime.datetime.today() - datetime.timedelta(days=0)
formatted_today = today.strftime("%d-%m-%Y")
user_jirayuth = auth.get_user("j1L0LVOQdLR2bJTWbgNCMikILX22")
user_tum = auth.get_user("RTn6E1WAjqW99UM0TMciWV50Dkh2")
user_kitikun = auth.get_user("zImuJrq1GNZuFbvdSOmGgFJbDzF2")

round = '2' ##รอบรถ2
ID33 = "ID33: คลัง Service Headoffice - Insure"
ID49INSURE = "ID49: สินค้า Trade-in"
ID49 = "ID49: สินค้าโยกเข้าคลัง"

data = firestore_db.collection('Scanned_ShipmentData').document(formatted_today).collection(user_jirayuth.uid).document(ID49).get() ###.document(user_jirayuth.uid).get()
doc_data = data.to_dict()[round]
df = pd.DataFrame.from_dict(doc_data)


In [36]:

EXCEL_READ_FILEPATH = r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\excel"
os.chdir(EXCEL_READ_FILEPATH)

new_wb = openpyxl.Workbook()
try:
    new_wb.save(f"Parcel Pickup {formatted_today} รอบที่ {round}.xlsx")
    
except FileExistsError:

    new_wb.save(f"Parcel Pickup {formatted_today} รอบที่ {round}.xlsx")

with pd.ExcelWriter(os.path.join(EXCEL_READ_FILEPATH, f"Parcel Pickup {formatted_today} รอบที่ {round}.xlsx"), mode="a", if_sheet_exists="replace", engine="openpyxl") as excel_file:
    df.to_excel(excel_file, index=False,)### startrow=excel_file.sheets[f"{formatted_today}"].max_row)
    excel_file.save()

In [ ]:
root = Tk()
load_json = filedialog.askopenfilename(title="เลือกไฟล์ JSON", filetypes={("*.json" , "JSON")})
data = pd.read_json(load_json)